<a href="https://colab.research.google.com/github/aas161/NeuroEvo-SANE/blob/main/SANE_NeuroEv_BMI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import csv
import os
import seaborn as sns
from sklearn.metrics import accuracy_score, f1_score, log_loss, precision_score, recall_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import networkx as nx

BMI Dataset URL: https://www.kaggle.com/datasets/yasserh/bmidataset/data

This data frame contains the following columns:

- Gender : Male (0) / Female (1);

- Height : Number (cm);

- Weight : Number (kg).

Index:

- 0 - Extremely Weak;

- 1 - Weak;

- 2 - Normal;

- 3 - Overweight;

- 4 - Obesity;

- 5 - Extreme Obesity.

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NeuroEvo/BMI_Dataset.csv')
df['Gender'] = df['Gender'].replace(['Male', 'Female'], [0, 1])

In [ ]:
df.head(8)

,Gender,Height,Weight,Index
0,0,174,96,4
1,0,189,87,2
2,1,185,110,4
3,1,195,104,3
4,0,149,61,3
5,0,189,104,3
6,0,147,92,5
7,0,154,111,5


In [ ]:
print("Количество примеров в каждом классе:\n\n", df.iloc[:,3:].value_counts())

Количество примеров в каждом классе:

 Index
5        198
4        130
2         69
3         68
1         22
0         13
Name: count, dtype: int64


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Gender  500 non-null    int64
 1   Height  500 non-null    int64
 2   Weight  500 non-null    int64
 3   Index   500 non-null    int64
dtypes: int64(4)
memory usage: 15.8 KB


In [ ]:
df.describe(percentiles=[0, .25, .30, .50, .75, .80, 1]).T.style.background_gradient(cmap = 'Reds')

,count,mean,std,min,0%,25%,30%,50%,75%,80%,100%,max
Gender,500.000000,0.510000,0.500401,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000
Height,500.000000,169.944000,16.375261,140.000000,140.000000,156.000000,159.000000,170.500000,184.000000,186.000000,199.000000,199.000000
Weight,500.000000,106.000000,32.382607,50.000000,50.000000,80.000000,84.000000,106.000000,136.000000,140.000000,160.000000,160.000000
Index,500.000000,3.748000,1.355053,0.000000,0.000000,3.000000,3.000000,4.000000,5.000000,5.000000,5.000000,5.000000


In [ ]:
# Разделение датасета на обучающую, тестовую и валидационную выборки
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
train_data, val_data = train_test_split(train_data, test_size=0.2, random_state=42)

# Разделение на признаки и целевую переменную
x_train = train_data.drop('Index', axis=1)
y_train = np.array(train_data['Index'])

x_test = test_data.drop('Index', axis=1)
y_test = np.array(test_data['Index'])

x_val = val_data.drop('Index', axis=1)
y_val = np.array(val_data['Index'])

In [ ]:
scaler = MinMaxScaler()

# Нормализация выборок в диапозоне от 0 до 1
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [ ]:
print('Тренировочная выборка:', x_train.shape)
print('Тестовая выборка:', x_test.shape)
print('Валидационная выборка:', x_val.shape)

Тренировочная выборка: (320, 3)
Тестовая выборка: (100, 3)
Валидационная выборка: (80, 3)


In [ ]:
# Параметры НС
epochs = 100
total_neurons = 3000
total_blueprints = 600
hidden_neurons = 100
in_neurons = x_train.shape[1]
out_neurons = np.unique(y_train).shape[0]
connections = 8
patience = 10
model_name = 1

model_path = f"/content/drive/MyDrive/Colab Notebooks/NeuroEvo/models/model_{model_name}.npy"